In [1]:
from sortedcontainers import SortedList as ost
from math import floor,ceil,log,sqrt
from collections import deque
import numpy as np
from statistics import mean,variance
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable

In [2]:
from dataclasses import dataclass

In [2]:
class normal_mean :
    def __init__(self,wsize) :
        if wsize != None : wsize += 1
        self.Y = deque([0.0],maxlen = wsize)
        self.SY = deque([0.0],maxlen = wsize)
        self.SYY = deque([0.0],maxlen = wsize)
    def push(self,y) :
        self.Y.append(y)
        self.SY.append(y + self.SY[-1])
        self.SYY.append(y*y + self.SYY[-1])
        # TODO - if Y is full - normalize
    def size(self) :
        return len(self.Y) - 1
    def __call__(self,a,b) :
        val = self.SY[b+1] - self.SY[a]
        val *= val
        val /= (b - a + 1)
        val = -val
        val += self.SYY[b+1] - self.SYY[a]
        return val

In [3]:
def class_left_right_sum(segment_cost_function) :
    def domain() : 
        return range(segment_cost_function.size())
    def value(i) :
        if i == 0 :
            return segment_cost_function(0,segment_cost_function.size()-1)
        else :
            return segment_cost_function(0,i-1) + segment_cost_function(i,segment_cost_function.size()-1)
    return (value,domain)

## detect a changepoint

In [4]:
def class_amoc(cost_function) :
    def update(value,beta) :
        cost_function.push(value)
        value,domain = class_left_right_sum(cost_function)
        cost,location = min([(value(i),i) for i in domain()])
        if value(0) - cost > beta :
            return cost,location
        return value(0),0
    return update

In [5]:
Z = list(np.random.normal(0, 1, 1000)) + list(np.random.normal(1, 1, 1000)) # + list(np.random.normal(0, 1, 1000))
mu = mean(Z)
sigma = sqrt(variance(Z))
Z = [float((z - mu)/sigma) for z in Z]
wsize = int(len(Z)/2)
beta = 2*log(wsize)

method = class_amoc(normal_mean(wsize))
iter = 0
for z in Z :
    value,location = method(z,beta)
    if location != 0 :
        msg = "change detected at iteration " + str(iter) + " located at " + str(location)
        #print(msg)
    iter += 1

## optimal partitioning

In [6]:
def fix_left(cost_function,position) :
    def value(i) :
        return cost_function(position,i)
    def size() :
        return cost_function.size()
    def domain(self) :
        return range(size())
    return(value,domain,size)

In [7]:
def fix_right(cost_function,position) :
    def value(i) :
        return cost_function(i,position)
    def size() :
        return cost_function.size()
    def domain(self) :
        return range(size())
    return(value,domain,size)

In [8]:
def left_right_sum(left,right) :
    value_l,domain,size = left
    value_r,_,_ = right
    def value(i) :
        if i == 0 :
            return value_r(i)
        else :
            return value_l(i) + value_r(i)
    return(value,domain,size)
    

In [9]:
fix_left = lambda f : (lambda i : f[0](0,i), f[1], f[2]) 
fix_right = lambda f : (lambda i : f[0](i,f[2]()-1), f[1], f[2]) 
left_right_sum = lambda l,r : lambda i : 

SyntaxError: invalid syntax (2388118503.py, line 3)

In [ ]:
f = lambda x : 2*x
g = lambda x : 3*x
h = lambda f : lambda x : f(x)


In [ ]:
t = h(f)

In [ ]:
t(2)

In [ ]:
f = lambda x : 5*x

In [ ]:
def class_left_right_sum(segment_cost_function) :
    def domain() : 
        return range(segment_cost_function.size())
    def value(i) :
        if i == 0 :
            return segment_cost_function(0,segment_cost_function.size()-1)
        else :
            return segment_cost_function(0,i-1) + segment_cost_function(i,segment_cost_function.size()-1)
    return (value,domain)

In [ ]:
class fix_left :
    def __init__(self,cost_function,position) :
        self.cost_function = cost_function
    def __call__(self,i) :
        return cost_function(position,i)
    def size(self) :
        return cost_function.size()
    def domain(self) :
        return range(self.size())

In [ ]:
class fix_right :
    def __init__(self,cost_function,position) :
        self.cost_function = cost_function
    def __call__(self,i) :
        return cost_function(i,position)
    def size(self) :
        return cost_function.size()
    def domain(self) :
        return range(self.size())

In [ ]:
def class_left_right_sum_new(left,right) :
    def domain(self) : 
        return range(left.size())
    def value(self.i) :
        if i == 0 :
            return right(0)
        else :
            return left(i-1) + right(i)
    return (value,domain)
    

In [ ]:
class history :
    def __init__(self,wsize) :
        self.wsize = wsize
        self.F = deque(maxlen = wsize)
    def push(self,val) :
        self.F.append(val)
        # TODO if F is full - normalize
    def __call__(self,i) :
        return self.F[i]
    def size(self) :
        return len(self.F)
    def domain(self) :
        return range(self.size())
        

In [ ]:
H = history(5)

In [ ]:
for i in range(10) :
    H.push((i,2*i))

In [ ]:
for i in H.domain() :
    print(H(i))

In [ ]:
from dataclasses import dataclass
from typing import Callable

In [3]:
@dataclass
class function :
    method : Callable[[int], float]
    domain : range
    size : int
    
    

In [36]:
@dataclass
class moments :
    y : float 
    sy : float
    syy : float

@dataclass
class sumstats :
    value : function
    domain : range
    size : int
    push : [[float],sumstats]    

def sumstats_factory(wsize) :
    stats = deque([moments(0.0,0.0,0.0)],maxlen = wsize)
    value = lambda i : stats[i]
    size = len(stats) 
    domain = range(size)
    def push(y) :
        s = stats[-1]
        stats.append(moments(s.y,s.sy + y,s.syy + y*y))
        return sumstats(value,domain,size,push)
    return sumstats(value,domain,size,push) 


In [ ]:
def 

In [46]:
blob = sumstats_factory(10)

In [47]:
blob.value(0)

moments(y=0.0, sy=0.0, syy=0.0)

In [48]:
glob = blob.push(3)

In [50]:
glob.value(1)

moments(y=0.0, sy=3.0, syy=9.0)

In [51]:
blob.value(1)

moments(y=0.0, sy=3.0, syy=9.0)

In [8]:
f = lambda x,y : x + y

In [9]:
blob = sumstats(f,range(10),11,f)

In [11]:
blob.value(3,4)

7

In [ ]:
def sumstats_factory(wsize) :
    stats = deque[sumstats(0.0,0.0,0.0),maxlen = wsize]
    value = lambda i : data[i]
    size = len(data) 
    domain = range(size)
    def push(y) :
        s = stats[-1]
        data.append(sumstats(stats.y,stats.sy + y,stats.syy + y*y))
        return
        
    return value,domain 
    

In [ ]:
s = sumstats(0,1,2)

In [ ]:
a,b,c = tuple(s)

In [ ]:
class normal_mean :
    def __init__(self,wsize) :
        if wsize != None : wsize += 1
        self.Y = deque([0.0],maxlen = wsize)
        self.SY = deque([0.0],maxlen = wsize)
        self.SYY = deque([0.0],maxlen = wsize)
    def push(self,y) :
        self.Y.append(y)
        self.SY.append(y + self.SY[-1])
        self.SYY.append(y*y + self.SYY[-1])
        # TODO - if Y is full - normalize
    def size(self) :
        return len(self.Y) - 1
    def __call__(self,a,b) :
        val = self.SY[b+1] - self.SY[a]
        val *= val
        val /= (b - a + 1)
        val = -val
        val += self.SYY[b+1] - self.SYY[a]
        return val

In [ ]:
f = lambda x,y : x + y

In [ ]:
blob = function(f,range(10),10)

In [ ]:
blob.method(3.1,7)

In [8]:
import itertools


In [9]:
x = deque([1,2,3])

In [10]:
iter(x)

In [ ]:
blob = [1,2,3]
iter(blob)